In [1]:
%matplotlib inline
# For command line argument parsing
import argparse

# For communication with evo_ros2
import zmq

# For data / fitness evaluation
import pandas as pd
from sklearn.metrics import mean_squared_error

# For DEAP
import random
from deap import creator, base, tools, algorithms
import numpy as np

# For email notifications
import smtplib
from email.mime.text import MIMEText

# For showing DEAP history plots
import matplotlib.pyplot as plt
import networkx




class DEAP_EA():
    def __init__(self, cmd_args):
        self.debug = cmd_args.debug
        
        
        # EA Params
        self.genome_size = 4
        self.tourn_size = 3
        self.pop_size = 2
        self.number_generations = 1
        
        
        # Socket Communication Params      
        self.ip_addr = '127.0.0.1'
        self.send_port = 5022
        self.recv_port = 5032

        # Email Notification Params
        self.email_receiver_list = ['glen.a.simon@gmail.com']
        
        
        # Set up 
        self.set_up_EA()
        self.set_up_sockets()
        
        
        # Run
        try:
            self.run()
        finally:
            self.socket.close()
            self.receiver.close()
            self.context.destroy()
        
    
    
    ### Run the EA ###
    def run(self):
        self.population = self.toolbox.population(n=self.pop_size)
        self.history.update(self.population)
        
        for gen in range(self.number_generations):
            offspring = algorithms.varAnd(self.population, self.toolbox, cxpb=0.5, mutpb=0.1)
            fits = self.toolbox.map(self.toolbox.evaluate, offspring)
            for fit, ind in zip(fits, offspring):
                ind.fitness.values = fit
            self.population = self.toolbox.select(offspring, k=len(self.population))


        top = tools.selBest(self.population, k=2)
        print(top)
        print(self.population)
        
        
        print(self.history.genealogy_tree)
        self.print_genealogy_tree()
        
        #self.email_notification(top)
    
    
    def print_genealogy_tree(self):
        graph = networkx.DiGraph(self.history.genealogy_tree)
        graph = graph.reverse()     # Make the grah top-down
        colors = [self.toolbox.evaluate(self.history.genealogy_history[i])[0] for i in graph]
        networkx.draw(graph, node_color=colors)
        plt.show()
        
    
    ### Set up evaluation function ###
    def evaluate_ind(self, ind):
        #print('Sending ind: {}'.format(ind))
        self.socket.send_json(ind)
        
        #print('Waiting Result')
        result = self.receiver.recv_json()
        
        #print('Recv\'d Result')
        df = pd.DataFrame.from_dict(dict(result))
        df['error'] = abs(df['Actual Speed'] - df['Goal Speed'])
        #df.plot(x='Time')
        fitness = mean_squared_error(df['Actual Speed'],  df['Goal Speed'])
        #print('Fitness: {}'.format(fitness))
        
        return (fitness, )
    
     
    ### Set up individual's shape, fitness function, and EA operators ###
    def set_up_EA(self):
        creator.create("FitnessMin", base.Fitness, weights=(-1.0,))
        creator.create("Individual", list, fitness=creator.FitnessMin)

        self.toolbox = base.Toolbox()
        self.toolbox.register("attr_float", random.random)
        self.toolbox.register("individual", tools.initRepeat, creator.Individual,
                 self.toolbox.attr_float, n=self.genome_size)
        self.toolbox.register("population", tools.initRepeat, list, self.toolbox.individual)
        
        # Set up Evo Algo operators
        self.toolbox.register("mate", tools.cxTwoPoint)
        self.toolbox.register("mutate", tools.mutGaussian, mu=0, sigma=1, indpb=0.2)
        self.toolbox.register("select", tools.selTournament, tournsize=self.tourn_size)
        self.toolbox.register("evaluate", self.evaluate_ind)
        
        # Set up EA history
        self.history = tools.History()
        
        # Decorate the variation operators
        self.toolbox.decorate("mate", self.history.decorator)
        self.toolbox.decorate("mutate", self.history.decorator)
        
        # Set up hall of fame and statistics
        self.hof = tools.HallOfFame(1)
        self.stats = tools.Statistics(lambda ind: ind.fitness.values)
        self.stats.register("avg", np.mean)
        self.stats.register("std", np.std)
        self.stats.register("min", np.min)
        self.stats.register("max", np.max)
        
        # Set up logbook
        self.logbook = tools.Logbook()


    ### Set up communication sockets ###
    def set_up_sockets(self):
        #Initialize the socket for data
        
        # Setup the socket to send data out on.
        self.context = zmq.Context()
        self.socket = self.context.socket(zmq.PUSH)
        #socket.setsockopt(zmq.LINGER, 0)    # discard unsent messages on close
        self.socket.bind('tcp://{}:{}'.format(self.ip_addr, self.send_port))

        # Setup the socket to read the responses on.
        self.receiver = self.context.socket(zmq.PULL)
        self.receiver.bind('tcp://{}:{}'.format(self.ip_addr, self.recv_port))

        # Setup ZMQ poller
        self.poller = zmq.Poller()
        self.poller.register(self.receiver, zmq.POLLIN)
       
    
    
    ### Email notification ###
    def email_notification(self, results):
        sender = 'evo.ros2.result.sender@gmail.com'
        msg = MIMEText(str(results))
        msg['From'] = sender
        msg['To'] = str(self.email_receiver_list)
        msg['Subject'] = 'EA Results'
    
        HOST = "smtp.gmail.com"
        PORT = "587"
        SERVER = smtplib.SMTP()
        SERVER.connect(HOST, PORT)
        USER = "evo.ros2.result.sender@gmail.com"
        PASSWD = "evoRos2Rocks"
        SERVER.ehlo()
        SERVER.starttls()
        SERVER.login(USER,PASSWD)

        #SERVER.set_debuglevel(True)
        SERVER.sendmail(sender, self.email_receiver_list, msg.as_string())
        SERVER.quit()



if __name__ == '__main__':
    # Parse arguments
    parser = argparse.ArgumentParser(description='Front end DEAP EA for PID Study')
    parser.add_argument('-d', '--debug', action='store_true', help='Print extra output to terminal.')
    args, unknown = parser.parse_known_args() # Only parse arguments defined above

    node = DEAP_EA(cmd_args = args)

KeyboardInterrupt: 